# Validate EIA-930 data against net generation outputs

In [ ]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from datetime import datetime
from dateutil.parser import parse as parse_dt
from datetime import timedelta
import json

import requests

In [ ]:
import sys

sys.path.append("../../src")

import filepaths

In [ ]:
year = 2020

In [ ]:
# EIA-930 data after timestamp adjustments but no cleaning
raw = pd.read_csv(
    f"{filepaths.data_folder()}/outputs/2020/eia930/eia930_raw.csv",
    index_col=0,
    parse_dates=True,
)

In [ ]:
GEN_ID = "EBA.{}-ALL.NG.H"
path = f"{filepaths.data_folder()}/results/{year}/power_sector_data/hourly/us_units/"
cors = {}
percent_difs = {}
annual_gen = {}
for ba_f in os.listdir(path):
    ba = ba_f.replace(".csv", "")
    print(ba, end="...")
    col_name = GEN_ID.format(ba)
    if col_name not in raw.columns:
        continue
    else:
        dat = pd.read_csv(path + ba_f, parse_dates=["datetime_utc"])
        dat = dat[dat.fuel_category == "total"]
        dat = dat.merge(raw[col_name], left_on="datetime_utc", right_index=True)
        c = dat[["net_generation_mwh", col_name]].corr().to_numpy()[0, 1]
        cors[ba] = c
        difs = (dat[col_name] - dat["net_generation_mwh"]) / dat["net_generation_mwh"]
        difs = difs.replace(np.inf, np.nan)
        percent_difs[ba] = difs.median()
        annual_gen[ba] = dat["net_generation_mwh"].sum()

In [ ]:
os.makedirs(
    f"{filepaths.data_folder()}/outputs/{year}/validation_metrics/us_units",
    exist_ok=True,
)

out = pd.DataFrame(
    data={
        "Difference as percent of hourly-egrid": percent_difs,
        "Correlation": cors,
        "Annual BA generation": annual_gen,
    }
)
out = out.sort_values("Annual BA generation", ascending=False)
out.to_csv(
    f"{filepaths.data_folder()}/outputs/{year}/validation_metrics/us_units/compare_930_hourlyegrid.csv"
)

# Visualize BA of interest

In [ ]:
ba = "NYIS"
col_name = GEN_ID.format(ba)
dat = pd.read_csv(path + ba + ".csv", parse_dates=["datetime_utc"])
dat = dat[dat.fuel_category == "total"]
dat = dat.merge(raw[col_name], left_on="datetime_utc", right_index=True)

px.line(dat, x="datetime_utc", y=["net_generation_mwh", col_name])

# Calculate real-time-rates from 930 + eGRID

In [ ]:
eia930 = pd.read_csv(
    f"{filepaths.data_folder()}/outputs/{year}/eia930/eia930_rolling.csv",
    parse_dates=True,
    index_col=0,
)

In [ ]:
## Load factors from Singularity API

# Use last year's egrid because that's all we have in real time
# TODO: could expand to other pollutants if we use eGRID download
url = f"https://api.singularity.energy/v1/emissions/"
egrid_year = str(year - 1)  # use last year as eGRID year

headers = {
    "X-Api-Key": os.environ["SINGULARITY_API_KEY"],
}

factors = {}

for adjustment in ["adjusted", "unadjusted"]:
    adjusted = adjustment == "adjusted"
    key = f"EGRID_{egrid_year}" if adjusted else f"EGRID_u{egrid_year}"
    response = requests.request("GET", url + key, headers=headers)
    factors[adjustment] = json.loads(response.content)["data"]

In [ ]:
# Default factors: coal factor is missing in FPC, PACW; so need national factor
default_factors = {}
default_factors["adjusted"] = {}
default_factors["unadjusted"] = {}
default_factors["adjusted"]["coal"] = 2168.237
default_factors["unadjusted"]["coal"] = 2168.237

In [ ]:
factors["adjusted"]["HST"]

In [ ]:
EIA_REGIONS = {
    "BPAT",
    "CISO",
    "ISNE",
    "MISO",
    "NYIS",
    "PJM",
    "SWPP",
}

In [ ]:
## For each BA, use singularity factors to calculate emission rate
bas_to_calc = [
    ba.replace(".csv", "")
    for ba in os.listdir(
        f"{filepaths.results_folder()}/2020/power_sector_data/hourly/us_units/"
    )
]

fuel_categories = {
    "coal": "COL",
    "natural_gas": "NG",
    "other": "OTH",
    "hydro": "WAT",
    "wind": "WND",
    "solar": "SUN",
    "nuclear": "NUC",
    "petroleum": "OIL",
}

for ba in bas_to_calc:
    singularity_ba = "EIA." + ba if ba in EIA_REGIONS else ba
    if singularity_ba not in factors[adjustment].keys():
        print(f"missing ba {singularity_ba}")
        continue

    out = pd.DataFrame(
        index=eia930.index,
        columns=[
            "adjusted_carbon",
            "unajusted_carbon",
            "adjusted_rate",
            "unadjusted_rate",
        ],
    )

    for adjustment in ["adjusted", "unadjusted"]:
        s_fuels = list(factors[adjustment][singularity_ba].keys())
        s_factors = [factors[adjustment][singularity_ba][f]["value"] for f in s_fuels]
        # Add default factors for missing fuel types
        for f in default_factors[adjustment].keys():
            if f not in s_fuels:
                s_fuels.append(f)
                s_factors.append(default_factors[adjustment][f])
        fuels = [fuel_categories[f] for f in s_fuels]
        generation_labels = [f"EBA.{ba}-ALL.NG.{f}.H" for f in fuels]

        out.loc[:, f"{adjustment}_carbon"] = (
            eia930[generation_labels].mul(s_factors, axis="columns").sum(axis="columns")
        )
        out.loc[:, f"{adjustment}_rate"] = (
            out.loc[:, f"{adjustment}_carbon"] / eia930.loc[:, f"EBA.{ba}-ALL.NG.H"]
        )

    os.makedirs(
        f"{filepaths.data_folder()}/outputs/{year}/validation/real_time_rate/",
        exist_ok=True,
    )
    out.to_csv(
        f"{filepaths.data_folder()}/outputs/{year}/validation/real_time_rate/{ba}.csv"
    )

# Rate: correlations and percent differences

Evaluation of rates

In [ ]:
gen_path = (
    f"{filepaths.data_folder()}/results/{year}/power_sector_data/hourly/us_units/"
)
consumed_path = (
    f"{filepaths.data_folder()}/results/{year}/carbon_accounting/hourly/us_units/"
)

In [ ]:
year = 2020

In [ ]:
percent_difs = {}
abs_difs = {}
med_rate = {}
cors = {}
max_difs = {}
for ba in os.listdir(
    f"{filepaths.data_folder()}/outputs/{year}/validation/real_time_rate/"
):
    if ba == ".DS_Store":  # just some os stuff
        continue
    ba = ba.replace(".csv", "")
    singularity_dat = pd.read_csv(
        f"{filepaths.data_folder()}/outputs/{year}/validation/real_time_rate/{ba}.csv",
        index_col=0,
        parse_dates=True,
    )
    # hourly_consumed = pd.read_csv(consumed_path+ba+".csv",
    #     usecols=["datetime_utc", "consumed_co2_rate_lb_per_mwh_for_electricity", "consumed_co2_rate_lb_per_mwh_adjusted"],
    #     index_col="datetime_utc", parse_dates=True)
    hourly_generated = pd.read_csv(
        gen_path + ba + ".csv",
        usecols=[
            "datetime_utc",
            "generated_co2_rate_lb_per_mwh_for_electricity",
            "generated_co2_rate_lb_per_mwh_for_electricity_adjusted",
            "co2_mass_lb",
            "fuel_category",
        ],
        index_col="datetime_utc",
        parse_dates=True,
    )
    hourly_generated = hourly_generated.loc[hourly_generated.fuel_category == "total"]
    hourly_generated = hourly_generated.sort_index()
    all_dat = pd.concat([singularity_dat, hourly_generated], axis="columns")

    dat_key = "generated_co2_rate_lb_per_mwh_for_electricity_adjusted"

    # Patch fix for PJM, see https://github.com/singularity-energy/open-grid-emissions/issues/230
    if ba == "PJM":
        all_dat.loc[all_dat[dat_key] < 100, dat_key] = np.nan
        all_dat = all_dat["2020-02-01T00:00":]

    # Patch fix for FPL real-time issue not caught by rolling filter
    if ba == "FPL":
        all_dat.loc[all_dat["adjusted_rate"] > 5000, "adjusted_rate"] = np.nan

    all_dat = all_dat.sort_index()
    cors[ba] = all_dat[[dat_key, "adjusted_rate"]].corr().to_numpy()[0, 1]
    percent_difs[ba] = (
        (all_dat["adjusted_rate"] - all_dat[dat_key]) / all_dat[dat_key]
    ).median()
    max_difs[ba] = (
        ((all_dat["adjusted_rate"] - all_dat[dat_key]) / all_dat[dat_key])
        .abs()
        .replace(1.0, np.nan)
        .max()
    )
    abs_difs[ba] = (all_dat["adjusted_rate"] - all_dat[dat_key]).median()
    med_rate[ba] = all_dat["adjusted_rate"].median()

In [ ]:
out = pd.DataFrame(
    data={
        "Median rate difference": abs_difs,
        "Difference as percent of OGE": percent_difs,
        "Correlation": cors,
        "Annual BA generation": annual_gen,
        "Median rate": med_rate,
    }
)
out = out.sort_values("Annual BA generation", ascending=False)

# Exclude BAs for which we couldn't calculate a real-time rate
todrop = [
    b
    for b in out.index
    if (b not in factors["adjusted"].keys())
    and ("EIA." + b not in factors["adjusted"].keys())
]
print(
    f"dropping {todrop} because they aren't included in Singularity's emission rate API"
)
out = out.drop(labels=todrop)
# exclude BAs for which rate is always zero (Hydro-only BAs)
zero_rates = []
for ba in out.index:
    if (out.loc[ba, "Median rate"] == 0) and (
        out.loc[ba, "Median rate difference"] == 0
    ):
        zero_rates.append(ba)
print(f"Note {zero_rates} have zero rates in OGE data")
# out = out.drop(labels=todrop)
# exclude BAs with zero net gen according to our data
zero_gen = []
for ba in out.index:
    if out.loc[ba, "Annual BA generation"] == 0:
        zero_gen.append(ba)
print(f"Dropping {zero_gen} because they have zero generation in OGE data")
out = out.drop(labels=zero_gen)

out.to_csv(
    f"{filepaths.data_folder()}/outputs/{year}/validation_metrics/us_units/compare_real_time_rates.csv"
)

In [ ]:
out.head()

In [ ]:
out_tbl = out.copy()  # .round(2)
out_tbl["Annual BA generation"] = (
    out_tbl["Annual BA generation"] / 1000000
)  # convert to millions
out_tbl["Difference as percent of OGE"] = (
    out_tbl["Difference as percent of OGE"] * 100
)  # convert to %
out_tbl = out_tbl.round(2)
for line in out_tbl.to_markdown().split("/n"):
    print(line)

In [ ]:
out.loc["FPC"]

# Visualize emission rate differences

In [ ]:
# For one-off interactive plotting
ba_of_interest = "BPAT"


real_time = pd.read_csv(
    f"{filepaths.data_folder()}/outputs/{year}/validation/real_time_rate/{ba_of_interest}.csv",
    index_col=0,
    parse_dates=True,
)
real_time = real_time["2020-01-01T00:00":]
if ba_of_interest == "NYIS":
    # NYIS has a hole in the EIA data that's not there in ISO data: fill it
    nyis_hole = pd.Series(
        data=[313, 287.79, 262.215],
        index=["2020-03-30T01:00+00", "2020-03-30T02:00+00", "2020-03-30T03:00+00"],
    )
    real_time.loc[nyis_hole.index, "adjusted_rate"] = nyis_hole

hourly_consumed = pd.read_csv(
    consumed_path + ba_of_interest + ".csv",
    usecols=[
        "datetime_utc",
        "consumed_co2_rate_lb_per_mwh_for_electricity",
        "consumed_co2_rate_lb_per_mwh_for_electricity_adjusted",
    ],
    index_col="datetime_utc",
    parse_dates=True,
)
hourly_generated = pd.read_csv(
    gen_path + ba_of_interest + ".csv",
    usecols=[
        "datetime_utc",
        "generated_co2_rate_lb_per_mwh_for_electricity",
        "generated_co2_rate_lb_per_mwh_for_electricity_adjusted",
        "co2_mass_lb",
        "fuel_category",
    ],
    index_col="datetime_utc",
    parse_dates=True,
)

all_dat = pd.concat(
    [
        real_time,
        hourly_consumed,
        hourly_generated.loc[hourly_generated.fuel_category == "total"],
    ],
    axis="columns",
)
all_dat = all_dat.sort_index()

all_dat["percent_difs"] = (
    all_dat["adjusted_rate"]
    - all_dat["generated_co2_rate_lb_per_mwh_for_electricity_adjusted"]
) / all_dat["generated_co2_rate_lb_per_mwh_for_electricity_adjusted"]

# all_dat = all_dat.loc[parse_dt("2020-07-19T00:00+00"):parse_dt("2020-08-06T00:00+00")]
# all_dat = all_dat.loc[parse_dt("2020-02-10T00:00+00"):parse_dt("2020-02-28T00:00+00")]

fig = px.line(
    all_dat,
    x=all_dat.index,
    y=["generated_co2_rate_lb_per_mwh_for_electricity_adjusted", "adjusted_rate"],
    title=f"Real time accuracy in {ba_of_interest}",
    labels={"value": "CO2 emission rate (lb/mwh)", "index": "Hour (UTC)"},
    template="plotly_white",
)

newnames = {
    "generated_co2_rate_lb_per_mwh_for_electricity_adjusted": "Historical benchmark",
    "adjusted_rate": "Real-time data",
}
fig.for_each_trace(lambda t: t.update(name=newnames[t.name]))
fig.update_layout(legend_title_text="")
fig.show()

os.makedirs(f"{filepaths.data_folder()}/outputs/viz/", exist_ok=True)
# pio.write_image(fig, f"{filepaths.data_folder()}/outputs/viz/{ba_of_interest}_aug_sm.jpg", width=1000*(2/3), height=500*(2/3), scale=3)

In [ ]:
factors["adjusted"]["EIA.NYIS"]

In [ ]:
### Plot natural gas emission rate: does this explain larger gap in summer?

hourly_rate = pd.read_csv(
    gen_path + ba_of_interest + ".csv",
    usecols=[
        "datetime_utc",
        "generated_co2_rate_lb_per_mwh_for_electricity",
        "generated_co2_rate_lb_per_mwh_for_electricity_adjusted",
        "co2_mass_lb",
        "fuel_category",
    ],
    index_col="datetime_utc",
    parse_dates=True,
)
hourly_rate = hourly_rate[hourly_rate.fuel_category == "natural_gas"]

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=hourly_rate.index,
        y=hourly_rate["generated_co2_rate_lb_per_mwh_for_electricity_adjusted"],
        name="Hourly emission rate",
    )
)
fig.add_trace(
    go.Scatter(
        x=[parse_dt("2020-01-01T00:00"), parse_dt("2021-01-01T00:00")],
        y=[
            factors["adjusted"]["EIA." + ba_of_interest]["natural_gas"]["value"],
            factors["adjusted"]["EIA." + ba_of_interest]["natural_gas"]["value"],
        ],
        name="eGRID annual emission rate",
        mode="lines",
    )
)

fig.update_xaxes(range=(parse_dt("2020-01-01T00:00"), parse_dt("2021-01-01T00:00")))
fig.update_layout(
    template="plotly_white",
    title=f"Natural gas emission rates in {ba_of_interest}O",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)

fig.update_yaxes(title_text="Natural gas emission rate<br>(lb CO2/MWh)")

fig.show()

pio.write_image(
    fig,
    f"{filepaths.data_folder()}/outputs/viz/gas_rate_{ba_of_interest}.jpg",
    width=1000 * (4 / 5),
    height=500 * (4 / 5),
    scale=3,
)

In [ ]:
oge_generation = pd.read_csv(
    gen_path + ba_of_interest + ".csv",
    usecols=["datetime_utc", "fuel_category", "net_generation_mwh"],
    index_col="datetime_utc",
    parse_dates=True,
)
oge_generation = oge_generation.pivot(
    columns="fuel_category", values="net_generation_mwh"
)

# plot real-time and OGE per-fuel generation in FPC to identify source of neg correlation
eiacols = [
    f"EBA.{ba_of_interest}-ALL.NG.COL.H",
    f"EBA.{ba_of_interest}-ALL.NG.NG.H",
    f"EBA.{ba_of_interest}-ALL.NG.NUC.H",
    f"EBA.{ba_of_interest}-ALL.NG.OIL.H",
    f"EBA.{ba_of_interest}-ALL.NG.OTH.H",
    f"EBA.{ba_of_interest}-ALL.NG.SUN.H",
    f"EBA.{ba_of_interest}-ALL.NG.UNK.H",
    f"EBA.{ba_of_interest}-ALL.NG.WAT.H",
    f"EBA.{ba_of_interest}-ALL.NG.WND.H",
]

toplot = pd.concat([eia930[eiacols], oge_generation])

In [ ]:
toplot.columns

In [ ]:
# plot real-time and OGE per-fuel generation in FPC to identify source of neg correlation
plotcols = [
    # f'EBA.{ba_of_interest}-ALL.NG.COL.H',
    # f'EBA.{ba_of_interest}-ALL.NG.NG.H',
    # f'EBA.{ba_of_interest}-ALL.NG.NUC.H',
    # f'EBA.{ba_of_interest}-ALL.NG.OIL.H',
    f"EBA.{ba_of_interest}-ALL.NG.OTH.H",
    # f'EBA.{ba_of_interest}-ALL.NG.SUN.H',
    f"EBA.{ba_of_interest}-ALL.NG.UNK.H",
    f"EBA.{ba_of_interest}-ALL.NG.WAT.H",
    # f'EBA.{ba_of_interest}-ALL.NG.WND.H',
    # "biomass",
    # "natural_gas",
    # "petroleum",
    # "solar",
    # "total",
    # "waste",
    # "geothermal",
    "hydro",
    # "wind",
]

px.line(toplot[plotcols])

In [ ]:
# What plants

In [ ]:
px.histogram(
    all_dat,
    x="percent_difs",
    title="NYIS hourly difference between benchmark and real-time<br>as percent of benchmark ",
)

# Roll up real-time to annual to compare to eGRID


In [ ]:
print(f"Real time aggregated over 2020, lb CO2 {all_dat['adjusted_carbon'].sum()}")
print(f"egrid is {28845962*2000}")

In [ ]:
(55539223793.10689 - 57691924000) / 57691924000

# Plot differences over BAs

Correlation, % difference, BA size, CI. 
Goal: show that errors are concentrated in smaller BAs 

In [ ]:
out.head()

In [ ]:
px.scatter(
    out,
    x="Difference as percent of OGE",
    y="Correlation",
    size="Annual BA generation",
    template="plotly_white",
)  # , text=out.index)

In [ ]:
# fig = px.scatter(out, x="Annual BA generation", y="Correlation", template="plotly_white")#, text=out.index)
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        y=[-3000000, 805000000],
        x=[1, 1],
        line={"width": 2, "color": "lightslategrey"},
        mode="lines",
    )
)
fig.add_trace(
    go.Scatter(
        y=out["Annual BA generation"],
        x=out["Correlation"],
        text=out.index,
        mode="markers",
        marker={"color": "rgb(17, 119, 51)"},
    )
)  # , color="Median rate")#, text=out.index)
fig.update_yaxes(range=(-3000000, 805000000))
fig.update_layout(template="plotly_white", showlegend=False)

fig.update_xaxes(dtick=0.250)
fig.show()
pio.write_image(
    fig,
    f"{filepaths.data_folder()}/outputs/viz/cor_ba_gen.jpg",
    width=800 * (1 / 2),
    height=900 * (1 / 2),
    scale=4,
)

In [ ]:
# px.scatter(out, x="Annual BA generation", y="Difference as percent of OGE")#, text=out.index)

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        y=[-3000000, 805000000],
        x=[0, 0],
        line={"width": 2, "color": "lightslategrey"},
        mode="lines",
    )
)
fig.add_trace(
    go.Scatter(
        y=out["Annual BA generation"],
        x=out["Median rate difference"],
        text=out.index,
        mode="markers",
        marker={"color": "rgb(17, 119, 51)"},
    )
)  # , color="Median rate")#, text=out.index)
fig.update_yaxes(range=(-3000000, 805000000))
fig.update_layout(template="plotly_white", showlegend=False)
fig.update_xaxes(dtick=500)
fig.show()
pio.write_image(
    fig,
    f"{filepaths.data_folder()}/outputs/viz/dif_ba_gen.jpg",
    width=800 * (1 / 2),
    height=900 * (1 / 2),
    scale=4,
)

In [ ]:
px.colors.qualitative.Safe[3]

# Plot natural gas emission rate as a "future directons" example

In [ ]:
# dat =

# Summary statistics

In [ ]:
good = len(out[out["Difference as percent of OGE"].abs() <= 0.1])
bad = len(out[out["Difference as percent of OGE"].abs() > 0.1])
print(good / (bad + good))

In [ ]:
for col in out.columns:
    out = out.replace(np.inf, np.nan)
    out = out.replace(-1 * np.inf, np.nan)
    non_nan_out = out.dropna(subset=col)
    a = np.average(non_nan_out[col].abs(), weights=non_nan_out["Annual BA generation"])
    print(f"{col} = {a}")

# Outputs

In [ ]:
# Plot and save all BAs
for ba_of_interest in os.listdir(
    f"{filepaths.data_folder()}/outputs/2020/validation/real_time_rate/"
):
    ba_of_interest = ba_of_interest.replace(".csv", "")
    if ".DS_" in ba_of_interest:
        continue

    real_time = pd.read_csv(
        f"{filepaths.data_folder()}/outputs/{year}/validation/real_time_rate/{ba_of_interest}.csv",
        index_col=0,
        parse_dates=True,
    )
    real_time = real_time["2020-01-01T00:00":]

    hourly_generated = pd.read_csv(
        gen_path + ba_of_interest + ".csv",
        usecols=[
            "datetime_utc",
            "generated_co2_rate_lb_per_mwh_for_electricity",
            "generated_co2_rate_lb_per_mwh_for_electricity_adjusted",
            "co2_mass_lb",
            "fuel_category",
        ],
        index_col="datetime_utc",
        parse_dates=True,
    )

    all_dat = pd.concat(
        [
            real_time,
            hourly_consumed,
            hourly_generated.loc[hourly_generated.fuel_category == "total"],
        ],
        axis="columns",
    )
    all_dat = all_dat.sort_index()

    fig = px.line(
        all_dat,
        x=all_dat.index,
        y=["generated_co2_rate_lb_per_mwh_for_electricity", "adjusted_rate"],
        title=f"{ba_of_interest} rate comparison",
        labels={"value": "Adjsuted CO2 emission rate (lb/mwh)", "index": "Hour"},
    )

    newnames = {
        "generated_co2_rate_lb_per_mwh_for_electricity": "Our data",
        "adjusted_rate": "Real-time data",
    }
    fig.for_each_trace(lambda t: t.update(name=newnames[t.name]))
    pio.write_image(
        fig,
        f"{filepaths.data_folder()}/outputs/viz/{ba_of_interest}.jpg",
        width=1000,
        height=400,
        scale=3,
    )